In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./cards'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Я нашел ограничитель так как у мен резко начал перегреваться комп от обучения модели,хотя я и все почистил и поменял термопасту я на всякий случай решил ограничить обучение

In [11]:
import tensorflow as tf

# Определяем процент использования ресурсов
limit = 0.7

# Ограничиваем использование памяти GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=int(gpu.memory_limit * limit))]
            )
    except RuntimeError as e:
        print(e)


In [2]:
file_path_test ="./cards.csv"
data = pd.read_csv(file_path_test, encoding='cp1251', )

In [3]:
data.head()

,class index,filepaths,labels,card type,data set
0,0,train/ace of clubs/001.jpg,ace of clubs,ace,train
1,0,train/ace of clubs/002.jpg,ace of clubs,ace,train
2,0,train/ace of clubs/003.jpg,ace of clubs,ace,train
3,0,train/ace of clubs/004.jpg,ace of clubs,ace,train
4,0,train/ace of clubs/005.jpg,ace of clubs,ace,train


In [55]:
for column in data.columns:
    unique_values = data[column].unique()
    print(f"Уникальные значения в столбце {column}: {unique_values}")

Уникальные значения в столбце class index: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52]
Уникальные значения в столбце filepaths: ['train/ace of clubs/001.jpg' 'train/ace of clubs/002.jpg'
 'train/ace of clubs/003.jpg' ... 'valid/two of spades/3.jpg'
 'valid/two of spades/4.jpg' 'valid/two of spades/5.jpg']
Уникальные значения в столбце labels: ['ace of clubs' 'ace of diamonds' 'ace of hearts' 'ace of spades'
 'eight of clubs' 'eight of diamonds' 'eight of hearts' 'eight of spades'
 'five of clubs' 'five of diamonds' 'five of hearts' 'five of spades'
 'four of clubs' 'four of diamonds' 'four of hearts' 'four of spades'
 'jack of clubs' 'jack of diamonds' 'jack of hearts' 'jack of spades'
 'joker' 'king of clubs' 'king of diamonds' 'king of hearts'
 'king of spades' 'nine of clubs' 'nine of diamonds' 'nine of hearts'
 'nine of spades' 'queen of clubs' 'queen of diamonds' 

In [4]:
# Находим индексы строк, где значение в столбце 'data set' равно 'train'
train_indices = np.where(data['data set'] == 'train')[0]

# Находим индексы строк, где значение в столбце 'data set' равно 'test'
test_indices = np.where(data['data set'] == 'test')[0]

# Находим индексы строк, где значение в столбце 'data set' равно 'valid'
valid_indices = np.where(data['data set'] == 'valid')[0]

# Используем найденные индексы для разделения датасета
train = data.iloc[train_indices]
test = data.iloc[test_indices]
valid = data.iloc[valid_indices]

In [7]:
for column in train.columns:
    unique_values = train[column].unique()
    print(f"Уникальные значения в столбце {column}: {unique_values}")
print("--------------------------------------------------------------------")
for column in test.columns:
    unique_values = test[column].unique()
    print(f"Уникальные значения в столбце {column}: {unique_values}")
print("--------------------------------------------------------------------")
for column in valid.columns:
    unique_values = valid[column].unique()
    print(f"Уникальные значения в столбце {column}: {unique_values}")

Уникальные значения в столбце class index: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52]
Уникальные значения в столбце filepaths: ['train/ace of clubs/001.jpg' 'train/ace of clubs/002.jpg'
 'train/ace of clubs/003.jpg' ... 'train/two of spades/153.jpg'
 'train/two of spades/154.jpg' 'train/two of spades/155.jpg']
Уникальные значения в столбце labels: ['ace of clubs' 'ace of diamonds' 'ace of hearts' 'ace of spades'
 'eight of clubs' 'eight of diamonds' 'eight of hearts' 'eight of spades'
 'five of clubs' 'five of diamonds' 'five of hearts' 'five of spades'
 'four of clubs' 'four of diamonds' 'four of hearts' 'four of spades'
 'jack of clubs' 'jack of diamonds' 'jack of hearts' 'jack of spades'
 'joker' 'king of clubs' 'king of diamonds' 'king of hearts'
 'king of spades' 'nine of clubs' 'nine of diamonds' 'nine of hearts'
 'nine of spades' 'queen of clubs' 'queen of diam

In [24]:
import cv2

# Загрузка изображения
image = cv2.imread('./cards/train/ace of clubs/001.jpg')

# Получение размерности изображения
image_shape = image.shape[:2]  # Получаем только ширину и высоту

print("Размерность изображения:", image_shape)

Размерность изображения: (224, 224)


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_folder = "./cards"

In [14]:
datagen = ImageDataGenerator(rescale=1./255)

In [17]:
# Чтение изображений из папки с использованием путей из датасета
image_data_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=image_folder,
    x_col='filepaths',  # Название столбца с путями к изображениям
    y_col='card type',       # Название столбца с метками
    target_size=(224, 224),  # Размер изображений
    batch_size=32,
    class_mode='categorical'  # Режим классификации
)

# Создание модели нейронной сети
model = tf.keras.models.Sequential([
    tf.keras.applications.MobileNetV2(include_top=False, input_shape=(224, 224, 3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(14, activation='softmax')  # Пример: 10 классов для классификации
])

# Компиляция модели
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(image_data_generator, epochs=10)"""

Found 7624 validated image filenames belonging to 14 classes.


c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="filepaths". These filename(s) will be ignored.
  warnings.warn(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


239/239 ━━━━━━━━━━━━━━━━━━━━ 438s 2s/step - accuracy: 0.5559 - loss: 1.3787
Epoch 2/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 419s 2s/step - accuracy: 0.8952 - loss: 0.3292
Epoch 3/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 418s 2s/step - accuracy: 0.9380 - loss: 0.1956
Epoch 4/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 420s 2s/step - accuracy: 0.9459 - loss: 0.1825
Epoch 5/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 418s 2s/step - accuracy: 0.9544 - loss: 0.1390
Epoch 6/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 428s 2s/step - accuracy: 0.9637 - loss: 0.1182
Epoch 7/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 426s 2s/step - accuracy: 0.9626 - loss: 0.1240
Epoch 8/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 427s 2s/step - accuracy: 0.9742 - loss: 0.0826
Epoch 9/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 426s 2s/step - accuracy: 0.9690 - loss: 0.0960
Epoch 10/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 432s 2s/step - accuracy: 0.9763 - loss: 0.0751


In [19]:
model.save("model_V2.h5")

ValueError: Unable to synchronously create dataset (name already exists)

In [20]:
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=image_folder,
    x_col='filepaths',
    y_col='card type',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Важно отключить перемешивание, чтобы сохранить соответствие между метками и предсказаниями
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid,
    directory=image_folder,
    x_col='filepaths',
    y_col='card type',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Важно отключить перемешивание, чтобы сохранить соответствие между метками и предсказаниями
)

# Оценка модели на тестовом наборе данных
test_metrics = model.evaluate(test_generator)

# Оценка модели на валидационном наборе данных
valid_metrics = model.evaluate(valid_generator)

print("Метрики на тестовом наборе данных:", test_metrics)
print("Метрики на валидационном наборе данных:", valid_metrics)

Found 265 validated image filenames belonging to 14 classes.
Found 265 validated image filenames belonging to 14 classes.


c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 335ms/step - accuracy: 0.5699 - loss: 3.0564
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 328ms/step - accuracy: 0.5581 - loss: 2.9119
Метрики на тестовом наборе данных: [3.1472280025482178, 0.5245283246040344]
Метрики на валидационном наборе данных: [3.112689256668091, 0.5132075548171997]


------------------------------------------------------------V-3--------------------------------------------------------------------------------

In [56]:
import cv2
import pandas as pd
import numpy as np

# Загрузка данных из CSV файла
data = pd.read_csv('./cards.csv')

# Предварительная обработка данных
images = []

for index, row in data.iterrows():
    # Загрузка изображения из файла
    image_path = row['filepaths']
    image = cv2.imread(image_path)
    
    # Масштабирование изображения до нужных размеров, если необходимо
    # image = cv2.resize(image, (desired_width, desired_height))
    
    # Преобразование изображения в матрицу
    image_matrix = np.array(image)
    
    # Добавление матрицы изображения в список
    images.append(image_matrix)

# Добавление нового столбца 'image' в датафрейм с матрицами изображений
data['image'] = images



In [57]:
data.head()

,class index,filepaths,labels,card type,data set,image
0,0,train/ace of clubs/001.jpg,ace of clubs,ace,train,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,0,train/ace of clubs/002.jpg,ace of clubs,ace,train,"[[[246, 248, 248], [249, 251, 251], [247, 249,..."
2,0,train/ace of clubs/003.jpg,ace of clubs,ace,train,"[[[0, 0, 0], [1, 1, 1], [1, 1, 1], [0, 0, 0], ..."
3,0,train/ace of clubs/004.jpg,ace of clubs,ace,train,"[[[255, 255, 252], [255, 255, 252], [252, 252,..."
4,0,train/ace of clubs/005.jpg,ace of clubs,ace,train,"[[[253, 255, 254], [250, 255, 253], [246, 254,..."


In [4]:
# Находим индексы строк, где значение в столбце 'data set' равно 'train'
train_indices = np.where(data['data set'] == 'train')[0]

# Находим индексы строк, где значение в столбце 'data set' равно 'test'
test_indices = np.where(data['data set'] == 'test')[0]

# Находим индексы строк, где значение в столбце 'data set' равно 'valid'
valid_indices = np.where(data['data set'] == 'valid')[0]

# Используем найденные индексы для разделения датасета
train = data.iloc[train_indices]
test = data.iloc[test_indices]
valid = data.iloc[valid_indices]

In [60]:
train.head()

,class index,filepaths,labels,card type,data set,image
0,0,train/ace of clubs/001.jpg,ace of clubs,ace,train,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,0,train/ace of clubs/002.jpg,ace of clubs,ace,train,"[[[246, 248, 248], [249, 251, 251], [247, 249,..."
2,0,train/ace of clubs/003.jpg,ace of clubs,ace,train,"[[[0, 0, 0], [1, 1, 1], [1, 1, 1], [0, 0, 0], ..."
3,0,train/ace of clubs/004.jpg,ace of clubs,ace,train,"[[[255, 255, 252], [255, 255, 252], [252, 252,..."
4,0,train/ace of clubs/005.jpg,ace of clubs,ace,train,"[[[253, 255, 254], [250, 255, 253], [246, 254,..."


In [20]:
train["image"].info

<bound method Series.info of 0       [[[255, 255, 255], [255, 255, 255], [255, 255,...
1       [[[246, 248, 248], [249, 251, 251], [247, 249,...
2       [[[0, 0, 0], [1, 1, 1], [1, 1, 1], [0, 0, 0], ...
3       [[[255, 255, 252], [255, 255, 252], [252, 252,...
4       [[[253, 255, 254], [250, 255, 253], [246, 254,...
                              ...                        
7620    [[[181, 183, 184], [183, 185, 186], [190, 192,...
7621    [[[255, 255, 255], [255, 255, 255], [254, 254,...
7622    [[[229, 229, 229], [235, 235, 235], [239, 239,...
7623    [[[238, 238, 238], [239, 239, 239], [241, 241,...
7624    [[[111, 109, 108], [138, 136, 135], [187, 185,...
Name: image, Length: 7625, dtype: object>

In [19]:
none_values = data[data['image'].isnull()]
print(none_values)

Empty DataFrame
Columns: [class index, filepaths, labels, card type, data set, image]
Index: []


IndexError: iloc cannot enlarge its target object

In [44]:
import numpy as np

# Создаем пустой список для хранения всех вариантов размерностей изображений
размерности_изображений = []

# Проходимся по каждому изображению в датасете и добавляем его размерность в список
for image in data['image']:
    if isinstance(image, np.ndarray):
        размерность = image.shape
        if размерность not in размерности_изображений:
            размерности_изображений.append(размерность)

# Выводим все варианты размерностей изображений
print("Все варианты размерностей изображений:")
for размерность in размерности_изображений:
    print(размерность)

Все варианты размерностей изображений:
(224, 224, 3)
()


In [45]:
# Создаем счетчик для подсчета пустых размерностей
количество_пустых_изображений = 0

# Проходимся по каждому изображению в датасете и проверяем его размерность
for image in data['image']:
    if isinstance(image, np.ndarray):
        if image.ndim == 0:  # Проверяем, имеет ли изображение пустую размерность
            количество_пустых_изображений += 1

# Выводим количество изображений с пустой размерностью
print("Количество изображений с пустой размерностью:", количество_пустых_изображений)

Количество изображений с пустой размерностью: 1


In [61]:
data_V34 = data[data['image'].apply(lambda x: isinstance(x, np.ndarray) and x.ndim != 0)]

# Выводим информацию о новом датасете
print("Новый размер датасета после удаления изображений с пустой размерностью:", data_V34.shape)

Новый размер датасета после удаления изображений с пустой размерностью: (8154, 6)


In [65]:
data_V34['image'] = data_V34['image'].apply(lambda x: x / 255.0 if isinstance(x, np.ndarray) else x)

MemoryError: Unable to allocate 1.15 MiB for an array with shape (224, 224, 3) and data type float64

In [62]:
data_V34.head()

,class index,filepaths,labels,card type,data set,image
0,0,train/ace of clubs/001.jpg,ace of clubs,ace,train,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,0,train/ace of clubs/002.jpg,ace of clubs,ace,train,"[[[246, 248, 248], [249, 251, 251], [247, 249,..."
2,0,train/ace of clubs/003.jpg,ace of clubs,ace,train,"[[[0, 0, 0], [1, 1, 1], [1, 1, 1], [0, 0, 0], ..."
3,0,train/ace of clubs/004.jpg,ace of clubs,ace,train,"[[[255, 255, 252], [255, 255, 252], [252, 252,..."
4,0,train/ace of clubs/005.jpg,ace of clubs,ace,train,"[[[253, 255, 254], [250, 255, 253], [246, 254,..."


In [49]:
data.head()

,class index,filepaths,labels,card type,data set,image
0,0,train/ace of clubs/001.jpg,ace of clubs,ace,train,"[[[6.030862941101084e-08, 6.030862941101084e-0..."
1,0,train/ace of clubs/002.jpg,ace of clubs,ace,train,"[[[5.8180089549445756e-08, 5.865309840757133e-..."
2,0,train/ace of clubs/003.jpg,ace of clubs,ace,train,"[[[0.0, 0.0, 0.0], [2.365044290627876e-10, 2.3..."
3,0,train/ace of clubs/004.jpg,ace of clubs,ace,train,"[[[6.030862941101084e-08, 6.030862941101084e-0..."
4,0,train/ace of clubs/005.jpg,ace of clubs,ace,train,"[[[5.983562055288528e-08, 6.030862941101084e-0..."


нормализовал данные

In [4]:
# Находим индексы строк, где значение в столбце 'data set' равно 'train'
train_indices = np.where(data['data set'] == 'train')[0]

# Находим индексы строк, где значение в столбце 'data set' равно 'test'
test_indices = np.where(data['data set'] == 'test')[0]

# Находим индексы строк, где значение в столбце 'data set' равно 'valid'
valid_indices = np.where(data['data set'] == 'valid')[0]

# Используем найденные индексы для разделения датасета
train = data.iloc[train_indices]
test = data.iloc[test_indices]
valid = data.iloc[valid_indices]

In [40]:
уникальные_значения = data["card type"].unique()
print(уникальные_значения)

['ace' 'eight' 'five' 'four' 'jack' 'xxx' 'king' 'nine' 'queen' 'seven'
 'six' 'ten' 'three' 'two']


У нас будет 14 классов 

In [5]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224)),  # Предполагается, что изображения уже нормализованы
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(14)  # 14 классов "card type"
])

# Компиляция модели
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Обучение модели
model.fit(train, epochs=10, validation_data=valid)

# Оценка модели
test_loss, test_acc = model.evaluate(test)
print('Точность на тестовых данных:', test_acc)

# Получение прогнозов модели
predictions = model.predict(test)
predicted_labels = tf.argmax(predictions, axis=1)

# Вычисление confusion matrix и classification report
true_labels = ...  # Истинные метки из тестовых данных
conf_matrix = confusion_matrix(true_labels, predicted_labels)
class_report = classification_report(true_labels, predicted_labels)

print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)

c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

До этого момента у мен был план вручную создать матрицы изобржений а потом их нормализовать,спустя 9 часов попыток оказалось что оперативной памяти у меня не хватает

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Создание генераторов данных для тренировочного, тестового и валидационного наборов
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Загрузка данных с использованием генераторов
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    x_col='filepaths',  # Путь к изображениям
    y_col='card type',  # Метки классов
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
      workers=2  # Для SparseCategoricalCrossentropy
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    x_col='filepaths',
    y_col='card type',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    workers=2
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid,
    x_col='filepaths',
    y_col='card type',
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    workers=2
)

# Создание и компиляция модели
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(14 , activation='softmax')  # 14 классов "card type"
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Обучение модели с использованием генераторов данных
history = model.fit(train_generator,
                    epochs=15,
                    validation_data=valid_generator)

# Оценка модели
test_loss, test_acc = model.evaluate(test_generator)
print('Точность на тестовых данных:', test_acc)

# Получение прогнозов модели
predictions = model.predict(test_generator)
predicted_labels = tf.argmax(predictions, axis=1)

# Вычисление confusion matrix и classification report
conf_matrix = confusion_matrix(true_labels, predicted_labels)
class_report = classification_report(true_labels, predicted_labels)

print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)


Found 7624 validated image filenames belonging to 14 classes.
Found 265 validated image filenames belonging to 14 classes.
Found 265 validated image filenames belonging to 14 classes.


c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="filepaths". These filename(s) will be ignored.
  warnings.warn(
c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


239/239 ━━━━━━━━━━━━━━━━━━━━ 122s 502ms/step - accuracy: 0.4110 - loss: 1.8897 - val_accuracy: 0.8264 - val_loss: 0.7045
Epoch 2/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 129s 540ms/step - accuracy: 0.7400 - loss: 0.8557 - val_accuracy: 0.8453 - val_loss: 0.5660
Epoch 3/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 125s 522ms/step - accuracy: 0.8130 - loss: 0.5776 - val_accuracy: 0.8868 - val_loss: 0.3920
Epoch 4/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 124s 519ms/step - accuracy: 0.9063 - loss: 0.3122 - val_accuracy: 0.9057 - val_loss: 0.5153
Epoch 5/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 113s 473ms/step - accuracy: 0.9484 - loss: 0.1867 - val_accuracy: 0.9094 - val_loss: 0.4133
Epoch 6/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 113s 471ms/step - accuracy: 0.9732 - loss: 0.1007 - val_accuracy: 0.9283 - val_loss: 0.3766
Epoch 7/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 115s 482ms/step - accuracy: 0.9860 - loss: 0.0623 - val_accuracy: 0.9208 - val_loss: 0.4523
Epoch 8/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 113s 472ms/step - accuracy: 0.9860 - loss: 0.06

NameError: name 'confusion_matrix' is not defined

In [13]:
model.save("model_V5.h5")

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
from numpy import sqrt, mean

# Затем вы можете использовать функцию mean для вычисления среднего значения
RMSE = sqrt(mean((true_labels - predicted_labels) ** 2))

# Вычисление confusion matrix и classification report
conf_matrix = confusion_matrix(true_labels, predicted_labels)
class_report = classification_report(true_labels, predicted_labels)


print('Confusion Matrix:\n', conf_matrix)
print('Classification Report:\n', class_report)
print('rmse:\n', RMSE)

Confusion Matrix:
 [[3 1 2 2 0 1 1 0 0 2 2 4 2 0]
 [3 2 0 0 0 2 1 1 3 1 3 3 1 0]
 [2 0 3 3 1 2 2 1 1 2 3 0 0 0]
 [2 2 2 0 1 2 2 1 2 2 1 2 1 0]
 [1 3 3 2 0 1 3 0 1 1 1 2 2 0]
 [2 3 0 2 2 0 1 0 2 2 2 2 1 1]
 [3 2 1 1 2 2 1 1 2 2 1 1 1 0]
 [1 1 2 1 2 2 1 4 1 1 0 0 3 1]
 [0 2 1 1 2 2 2 2 2 3 2 1 0 0]
 [1 1 1 1 1 5 1 2 1 1 1 2 2 0]
 [1 1 3 2 3 0 1 3 2 0 0 1 3 0]
 [2 2 0 2 3 1 2 2 2 1 2 1 0 0]
 [3 1 0 1 1 2 1 3 2 3 1 2 0 0]
 [0 0 0 0 0 1 2 1 0 1 0 0 0 0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.12      0.15      0.14        20
           1       0.10      0.10      0.10        20
           2       0.17      0.15      0.16        20
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00        20
           5       0.00      0.00      0.00        20
           6       0.05      0.05      0.05        20
           7       0.19      0.20      0.20        20
           8       0.10      0.10  

In [22]:
# Вычисление матрицы ошибок
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Полнота (Recall) вычисляется как TP / (TP + FN)
TP = conf_matrix[1, 1]  # Количество истинно положительных предсказаний
FN = conf_matrix[1, 0]  # Количество ложно отрицательных предсказаний
recall = TP / (TP + FN)

# F1-мера (F1-Score) вычисляется как гармоническое среднее между точностью и полнотой
precision = TP / (TP + conf_matrix[0, 1])  # Точность (Precision) = TP / (TP + FP)
f1_score = 2 * (precision * recall) / (precision + recall)

# Вывод результатов
print("Полнота (Recall):", recall)
print("F1-мера (F1-Score):", f1_score)

Полнота (Recall): 0.4
F1-мера (F1-Score): 0.5
